In [3]:
!pip install folium

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
%matplotlib inline

pd.set_option("display.max_columns",None)

import warnings
warnings.filterwarnings("ignore")

  Using cached folium-0.14.0-py2.py3-none-any.whl (102 kB)
  Using cached branca-0.6.0-py3-none-any.whl (24 kB)


In [ ]:
# Load and check base data files from public S3 bucket
%store -r s3_private_path

In [5]:


fires_df = pd.read_csv(s3_private_path+'fires.csv', parse_dates=True)
weather_df = pd.read_csv(s3_private_path+'weather.csv', parse_dates=True)
conditions_df = pd.read_csv(s3_private_path+'conditions.csv', parse_dates=True)

FileNotFoundError: sagemaker-us-east-1-857283526476/widfire-risk/data/csv/fires.csv

In [ ]:
fires_df.head()

In [ ]:
weather_df.head()

In [ ]:
conditions_df.head()

In [ ]:
fires = fires_df.copy()
weather = weather_df.copy()
conditions = conditions_df.copy()

In [ ]:
fires.shape 

In [ ]:
weather.shape

In [ ]:
conditions.shape

In [ ]:
fires.info()

In [ ]:
weather.info()

In [ ]:
conditions.info()

In [ ]:
fires.nunique()

In [ ]:
fires.columns

In [ ]:
weather.nunique()

In [ ]:
conditions.nunique()

In [ ]:
fires.describe()

In [ ]:
weather.describe()

In [ ]:
conditions.describe()

In [ ]:
fires['ContainmentDateTime'] = pd.to_datetime(fires['ContainmentDateTime'])
fires['ControlDateTime'] = pd.to_datetime(fires['ControlDateTime'])
fires['EstimatedCostToDate'] = pd.to_datetime(fires['EstimatedCostToDate'])
fires['FireDiscoveryDateTime'] = pd.to_datetime(fires['FireDiscoveryDateTime'])
fires['FireOutDateTime'] = pd.to_datetime(fires['FireOutDateTime'])

In [ ]:
print(fires.dtypes)

In [ ]:
# fires.filter(like='Date').apply(pd.to_datetime)

# print(fires.dtypes)

In [ ]:
fires.isnull().sum() 

In [ ]:
weather.isnull().sum()

In [ ]:
weather['DATE'] = pd.to_datetime(weather['DATE'])

In [ ]:
print(weather.dtypes)

In [ ]:
fires.groupby('FireCause').size()

In [ ]:
sns.countplot(x = 'FireCause', data=fires)

In [ ]:
numerical_features_w = weather.select_dtypes(include=["number"])
numerical_features_w = numerical_features_w.drop(["LONGITUDE", "LATITUDE"], axis=1)
numerical_features_w.head()

In [ ]:
numerical_features_f = fires.select_dtypes(include=["number"])
numerical_features_f = numerical_features_f.drop(["InitialLongitude", "InitialLatitude"], axis=1)
numerical_features_f.head()

In [ ]:
# Extract the year from the FireDiscoveryDateTime column
fires['Year'] = fires['FireDiscoveryDateTime'].dt.year
fires['month'] = fires['FireDiscoveryDateTime'].dt.month
fires['day'] = fires['FireDiscoveryDateTime'].dt.day  

# Count the number of fires in each year
fires_by_year = fires['Year'].value_counts()

# Create a histogram of the number of fires in each year
plt.hist(fires['Year'], bins=len(fires_by_year))
plt.xlabel('Year')
plt.ylabel('Number of fires')
plt.show()

In [ ]:
# Drop rows with missing latitude or longitude values
fires = fires.dropna(subset=['InitialLatitude', 'InitialLongitude'])

In [ ]:
# Create a map centered on the US
mapb = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Add a marker for each fire location
# for index, row in fires.iterrows():
#     folium.Marker([row['InitialLatitude'], row['InitialLongitude']]).add_to(map)

# Show the map
# map

In [ ]:
# Fire damage

plt.scatter(fires['InitialLongitude'], fires['InitialLatitude'], s=fires['FinalAcres'], alpha=0.5)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Wildfires by Location and Size')
plt.show()

In [ ]:
# Box Plot of Fire Duration
fires['duration'] = fires['FireOutDateTime'] - fires['FireDiscoveryDateTime']
plt.boxplot(fires['duration'].dt.days.dropna())
plt.ylabel('Duration in days')
plt.title('Wildfire Duration')
plt.show()

In [ ]:
# Scatterplot to show cost vs acres of damage
subset = fires[['EstimatedCostToDate', 'FinalAcres']]

subset = subset.dropna()

# Create scatter plot
plt.scatter(subset['FinalAcres'], subset['EstimatedCostToDate'])
plt.xlabel('Final Acres')
plt.ylabel('Estimated Cost To Date')
plt.title('Relationship Between Wildfire Size and Cost')
plt.show()

In [ ]:
# Stacked bar graph to see wildfires by month and any seasonal patterns between the causes
subset = fires[['FireCause', 'FireDiscoveryDateTime']]

subset = subset.dropna()

grouped = subset.groupby([subset['FireDiscoveryDateTime'].dt.month, 'FireCause']).size().unstack()

grouped.plot(kind='bar', stacked=True)
plt.xlabel('Month')
plt.ylabel('Number of Wildfires')
plt.title('Number of Wildfires by Month and Cause')
plt.show()

In [ ]:
# Drop rows with missing latitude or longitude values
weather = weather.dropna(subset=['LATITUDE', 'LONGITUDE'])

In [ ]:
fires['FireDiscoveryDateTime'] = fires['FireDiscoveryDateTime'].dt.tz_localize(None) #remove timezone

In [ ]:
merged_df = pd.merge(fires, weather, how='left', left_on=['FireDiscoveryDateTime', 'InitialLongitude', 'InitialLatitude'], right_on=['DATE', 'LONGITUDE', 'LATITUDE'])

In [ ]:
merged_df.head()